In [ ]:
import warnings
warnings.filterwarnings("ignore")
import arviz as az
import pandas as pd
import plotly.express as px
import numpy as np
from tbdynamics.camau.calibration.utils import get_bcm
from tbdynamics.calibration.plotting import plot_output_ranges, plot_trial_output_ranges
from tbdynamics.tools.inputs import load_targets
from tbdynamics.settings import CM_PATH, OUT_PATH, DOCS_PATH
from tbdynamics.constants import QUANTILES
from tbdynamics.camau.constants import indicator_legends, indicator_names
import estival.sampling.tools as esamp
from typing import Dict, Optional
from tbdynamics.tools.detect import make_future_acf_scenarios

In [ ]:
# loaded_inference_data = az.from_netcdf(OUT_PATH / 'inference_data1.nc')
# idata = az.from_netcdf(OUT_PATH / 'extracted_idata.nc')
params = {
    "start_population_size": 30000.0,
    "seed_time": 1805.0,
    "seed_num": 1.0,
    "seed_duration": 1.0,
    # "contact_rate": 0.02,
    "rr_infection_latent": 0.1890473700762809,
    "rr_infection_recovered": 0.17781844797545143,
    "smear_positive_death_rate": 0.3655528915762244,
    "smear_negative_death_rate": 0.027358324164819155,
    "smear_positive_self_recovery": 0.18600338108638945,
    "smear_negative_self_recovery": 0.11333894801537307,
    "screening_scaleup_shape": 0.3,
    "screening_inflection_time": 1993,
    # "time_to_screening_end_asymp": 2.1163556520843936,
    "acf_sensitivity": 0.90,
    # "prop_mixing_same_stratum": 0.6920672992582717,
    # "early_prop_adjuster": -0.017924441638418186,
    # "late_reactivation_adjuster": 1.1083422207175728,
    # "total_population_dispersion": 3644.236227852164,
    # "notif_dispersion": 88.37092488550051,
    # "latent_dispersion": 7.470896188551709,
}
targets = load_targets(CM_PATH / "targets.yml")

In [ ]:
idata_raw = az.from_netcdf(OUT_PATH / 'camau/best/calib_full_out.nc')

In [ ]:
burnt_idata = idata_raw.sel(draw=np.s_[5000:])
idata_extract = az.extract(burnt_idata, num_samples=300)

In [ ]:
# outputs = calculate_scenario_outputs(params, idata)
# with open(OUT_PATH / 'quant_outputs.pkl', 'wb') as f:
#      pickle.dump(outputs, f)

In [ ]:
# with open(OUT_PATH /'camau/quant_outputs.pkl', 'rb') as f:
#     outputs = pickle.load(f)
covid_effects = {"detection_reduction": True, "contact_reduction": False}

#     # Base scenario (calculate outputs for all indicators)
bcm = get_bcm(params, covid_effects)
base_results = esamp.model_results_for_samples(idata_extract, bcm).results
base_quantiles = esamp.quantiles_for_results(base_results, QUANTILES)

In [ ]:
# base_quantiles.to_pickle(OUT_PATH / 'camau/output0304.pkl')
# base_quantiles = pd.read_pickle(OUT_PATH / 'camau/output0304.pkl')

In [ ]:
# target_plot.write_image(DOCS_PATH / "targets1.png", scale=3)
plot_output_ranges(base_quantiles,targets,["total_population","act3_trial_adults_pop", "act3_control_adults_pop"],indicator_names,indicator_legends,1,2010,2025, option = 'camau') #.write_image(DOCS_PATH /'camau/pops.png', scale=3)

In [ ]:
plot_output_ranges(base_quantiles,targets,['notification','percentage_latent_adults'],indicator_names,indicator_legends,2,2010,2025, option='camau') #.write_image(DOCS_PATH /'camau/targets.png', scale=3)


In [ ]:
plot_output_ranges(base_quantiles,targets,['incidence', 'prevalence_pulmonary', 'adults_prevalence_pulmonary','mortality'],indicator_names,indicator_legends,2,2010,2025, option='camau') #.write_image(DOCS_PATH /'camau/compare.png', scale=3)

In [ ]:
plot_output_ranges(base_quantiles,targets,['detection_rate'],indicator_names,indicator_legends,1,1980,2025)

In [ ]:
plot_output_ranges(base_quantiles,targets,['incidence', 'prevalence_pulmonary', 'adults_prevalence_pulmonary','mortality'],indicator_names,indicator_legends,2,2010,2025, option='camau')

In [ ]:
plot_trial_output_ranges(base_quantiles,targets,['acf_detectionXact3_trial','acf_detectionXact3_control'],indicator_names,2) #.write_image(DOCS_PATH /'camau/trial.png', scale=3)

In [ ]:
arms = ['act3_trial', 'act3_control', 'act3_other']
metrics = ['incidenceX', 'prevalence_infectiousX']
indicators = [f"{metric}{arm}" for arm in arms for metric in metrics]
plot_output_ranges(
    base_quantiles,
    targets,
    indicators,
    indicator_names,
    indicator_legends,
    2,
    2010,
    2025,
    option='camau',
) #.write_image(DOCS_PATH / 'camau/output_arms.png', scale=3)

In [ ]:
# target_plot.write_image(DOCS_PATH / "targets2.png", scale=3)

In [ ]:
# spah.write_image(DOCS_PATH / 'spah.png', scale = 3)

In [ ]:
# target_plot_history = plot_output_ranges(outputs['base_scenario'],targets,['total_population','notification','adults_prevalence_pulmonary'],1,1800,2010, history =True)

In [ ]:
# target_plot_history

In [ ]:
# target_plot_history.write_image(DOCS_PATH / 'targets_history.png', scale=3)

In [ ]:
# compare_target_plot = plot_output_ranges(outputs['base_scenario'],targets,['incidence','mortality_raw','prevalence_smear_positive', 'percentage_latent'],2,2010,2025)

In [ ]:
# compare_target_plot.write_image(DOCS_PATH / "non_targets.png", scale='3')

In [ ]:
# compare_target_plot

In [ ]:
# screening_plot.write_image(DOCS_PATH / 'screening_plot.png', scale =3)

In [ ]:
# cdr_plot = plot_output_ranges(outputs['base_scenario']['quantiles'],targets,['case_notification_rate'],1,2010,2025)

In [ ]:
# cdr_plot.write_image(DOCS_PATH / 'cdr_plot.png', scale =3)

In [ ]:
# early_plot = plot_output_ranges(base_quantiles,targets,['incidence_early_prop'], indicator_names, indicator_legends,1,2000,2025) #.write_image(DOCS_PATH /'camau/early.png', scale=3)

In [ ]:
def calculate_future_acf_outputs(
    params: Dict[str, float],
    idata_extract: az.InferenceData,
    covid_effects: Optional[Dict[str, bool]] = None,
    future_acf_scenarios: Dict[str, Dict[float, float]]=None,
) -> Dict[str, pd.DataFrame]:
    """
    Calculate the model outputs for future ACF scenarios.

    Args:
        params: Dictionary containing model parameters.
        idata_extract: InferenceData object containing the model data.
        future_acf_scenarios: Dictionary of future ACF scenarios with their parameters.

    Returns:
        A dictionary containing the results for each future ACF scenario.
    """
    future_outputs = {}

    for scenario_name, scenario_params in future_acf_scenarios.items():
        # Get the model results for the current scenario
        bcm = get_bcm(params, covid_effects, future_acf_scenarios={scenario_name: scenario_params})
        base_results = esamp.model_results_for_samples(idata_extract, bcm).results
        base_quantiles = esamp.quantiles_for_results(base_results, QUANTILES)

        # Extract relevant indicators
        # indicators = ["notification", "total_population", "adults_prevalence_pulmonary"]
        future_outputs[scenario_name] = base_quantiles

    return future_outputs

In [ ]:
config = {
    "arm": ["trial"],
    "every": [2,4],
    "coverage": [0.5, 0.8],
}
future_acf_scenarios = make_future_acf_scenarios(config)

In [ ]:
future_acf_scenarios

In [ ]:
outputs = calculate_future_acf_outputs(params, idata_extract,covid_effects, future_acf_scenarios)

In [ ]:
outputs['trial_2_80']['act3_trial_adults_prevalence'].loc[2025:2036].plot()

In [ ]:
outputs['trial_4_80']['act3_trial_adults_prevalence'].loc[2025:2036].plot()

In [ ]:
base_quantiles['act3_trial_adults_prevalence'].loc[2025:2036].plot()